In [ ]:
from paddleocr import PaddleOCR, draw_ocr
import cv2
import matplotlib.pyplot as plt

# Initialize PaddleOCR with GPU support
ocr_model = PaddleOCR(
    use_angle_cls=True,
    lang='en',
    use_gpu=True  # This ensures GPU is used
)

# Path to the vehicle number plate image
image_path = r"P:\jindal project\cropped_img\63.jpg_plate_20250320_140555_0.png"

# Perform OCR
result = ocr_model.ocr(image_path)

# Extract and print the text
extracted_text = ' '.join([line[1][0] for line in result[0]])
print("Extracted Number Plate Text:", extracted_text.strip())

# Visualize the results
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

boxes = [line[0] for line in result[0]]
txts = [line[1][0] for line in result[0]]
scores = [line[1][1] for line in result[0]]

# Draw OCR boxes and text
img_with_boxes = draw_ocr(image, boxes, txts, scores, font_path=r"C:\Users\amarn\Downloads\Open_Sans\OpenSans-VariableFont_wdth,wght.ttf")

# Display image
plt.figure(figsize=(10, 10))
plt.imshow(img_with_boxes)
plt.axis('off')
plt.show()


In [ ]:
from paddleocr import PaddleOCR, draw_ocr
import cv2
import re
import matplotlib.pyplot as plt

# Initialize OCR with GPU support and English recognition
ocr_model = PaddleOCR(use_angle_cls=True, lang='en', use_gpu=True)

# Path to video file
video_path = r"C:\Users\amarn\myenv\Scripts\truck.mp4"

# Open video
cap = cv2.VideoCapture(video_path)

# Font path for drawing text
font_path = r"C:\Users\amarn\Downloads\Open_Sans\OpenSans-VariableFont_wdth,wght.ttf"

# Process every Nth frame
frame_skip = 10  # Change this to 1 for every frame, 10 for every 10th frame
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if frame_count % frame_skip == 0:
        # Convert frame to RGB (PaddleOCR expects RGB)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Run OCR
        result = ocr_model.ocr(rgb_frame)

        # Print result for debugging
        print("OCR Result:", result)

        # Check if OCR result is valid
        if result and result[0]:
            text_segments = [line[1][0] for line in result[0]]
            raw_text = ''.join(text_segments).upper()
            pattern = r"[A-Z]{2}[0-9]{2}[A-Z]{1,2}[0-9]{3,4}"
            match = re.search(pattern, raw_text)
            number_plate = match.group() if match else "Not Detected"

            # Draw OCR results
            boxes = [line[0] for line in result[0]]
            txts = [line[1][0] for line in result[0]]
            scores = [line[1][1] for line in result[0]]
            annotated_img = draw_ocr(rgb_frame, boxes, txts, scores, font_path=font_path)
            annotated_img = cv2.cvtColor(annotated_img, cv2.COLOR_RGB2BGR)

            # Display with plate number as title
            cv2.putText(annotated_img, f"Plate: {number_plate}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.imshow("Number Plate Detection", annotated_img)

            # Exit on ESC key
            if cv2.waitKey(1) & 0xFF == 27:
                break

    frame_count += 1

cap.release()
cv2.destroyAllWindows()


In [1]:
import os
import cv2
from ultralytics import YOLO

# Input and output paths
video_path = r"C:\Users\amarn\myenv\Scripts\truck.mp4"
video_out_path = r"C:\Users\amarn\myenv\Scripts\cropped_plate_video.mp4"

# Load YOLO model
model = YOLO(r"C:\Users\amarn\myenv\Scripts\runs\detect\train6\weights\best.pt")

cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()

# Define video writer later after detecting first plate
out = None
frame_count = 0
threshold = 0.5

while ret:
    results = model(frame)[0]

    cropped = None
    for result in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result
        if score > threshold:
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
            cropped = frame[y1:y2, x1:x2]
            break  # Take first detection only

    if cropped is not None:
        # Initialize video writer once
        if out is None:
            H, W = cropped.shape[:2]
            out = cv2.VideoWriter(video_out_path, cv2.VideoWriter_fourcc(*'MP4V'),
                                  int(cap.get(cv2.CAP_PROP_FPS)), (W, H))

        out.write(cropped)

    ret, frame = cap.read()
    frame_count += 1

cap.release()
if out:
    out.release()



0: 384x640 1 numberplate, 48.0ms
Speed: 2.0ms preprocess, 48.0ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 numberplate, 38.3ms
Speed: 1.4ms preprocess, 38.3ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 numberplate, 33.0ms
Speed: 1.0ms preprocess, 33.0ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 numberplate, 33.5ms
Speed: 0.8ms preprocess, 33.5ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 numberplate, 31.6ms
Speed: 0.9ms preprocess, 31.6ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 numberplate, 32.8ms
Speed: 0.8ms preprocess, 32.8ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 numberplate, 33.0ms
Speed: 0.9ms preprocess, 33.0ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 numberplate, 32.6ms
Speed: 0.9ms preprocess, 32.6ms inference, 0.4ms 

In [2]:
from paddleocr import PaddleOCR, draw_ocr
import cv2
import re
import csv
import os

# Initialize OCR with GPU support
ocr_model = PaddleOCR(use_angle_cls=True, lang='en', use_gpu=True)

# Paths
video_path = r"C:\Users\amarn\myenv\Scripts\cropped_plate_video.mp4"
font_path = r"C:\Users\amarn\Downloads\Open_Sans\OpenSans-VariableFont_wdth,wght.ttf"
csv_output_path = r"C:\Users\amarn\myenv\Scripts\extracted_plates.csv"

# Create CSV file and write headers
with open(csv_output_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Frame Number", "Extracted Text"])

    cap = cv2.VideoCapture(video_path)
    frame_skip = 1
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_skip == 0:
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            result = ocr_model.ocr(rgb_frame)

            print("OCR Result:", result)

            number_plate = "Not Detected"
            if result and result[0]:
                text_segments = [line[1][0] for line in result[0]]
                raw_text = ''.join(text_segments).upper()
                pattern = r"[A-Z]{2}[0-9]{2}[A-Z]{1,2}[0-9]{3,4}"
                match = re.search(pattern, raw_text)
                number_plate = match.group() if match else "Not Detected"

                # Draw OCR annotations
                boxes = [line[0] for line in result[0]]
                txts = [line[1][0] for line in result[0]]
                scores = [line[1][1] for line in result[0]]

                from paddleocr import draw_ocr
                annotated_img = draw_ocr(rgb_frame, boxes, txts, scores, font_path=font_path)
                annotated_img = cv2.cvtColor(annotated_img, cv2.COLOR_RGB2BGR)

                cv2.putText(annotated_img, f"Plate: {number_plate}", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv2.imshow("Number Plate Detection", annotated_img)

                if cv2.waitKey(1) & 0xFF == 27:
                    break

            # Write to CSV
            writer.writerow([frame_count, number_plate])

        frame_count += 1

    cap.release()
    cv2.destroyAllWindows()


c:\Program Files\Python310\lib\site-packages\paddle\utils\cpp_extension\extension_utils.py:711: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)


[2025/04/03 23:49:16] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, use_gcu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\amarn/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\amarn/.paddleocr/whl\\rec\\en\\en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320',